# **5. Pytorch를 이용한 xor 구현**

목표 : 라이브러리 pytorch를 이용하여 그동안 했던 xor 신경망을 구현해봅시다.
---

In [ ]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 가중치 초기화 함수
def weight_init(m):
    classname = m.__class__.__name__
    # m에서 classname이 Linear(신경망 레이어)인 경우
    if classname.find('Linear') != -1:
        # weight를 uniform distribution을 이용하여 초기화하고 bias는 0으로 초기화
        m.weight.data.uniform_(0.0, 1.0)
        m.bias.data.fill_(0)

# Xor data를 만들고 모델에 넣어줄 데이터로더. torch.utils.data.Dataset를 상속받고 __len__과 __getitem__을 선언해준다.
class xor_dataloader(torch.utils.data.Dataset):
    # 신경망 학습 중 신경망에 데이터를 공급해주는 dataloader 정의
    def __init__(self):
        super().__init__()
        # 외부 파일을 이용할 경우 여기서 파일을 읽거나 파일들의 경로를 찾음
        self.input = torch.Tensor([[0, 0], [0, 1], [1, 0], [1, 1]])
        self.target = torch.Tensor([[0], [1], [1], [0]])

    def __len__(self):
        # dataloder로 기능하기위해 선언 필요.
        # 데이터의 총 개수가 출력되도록 함
        return len(self.input)

    def __getitem__(self, item):
        # dataloader에 데이터를 요청하였을 때 다음 데이터를 제공.
        X = self.input[item]
        t = self.target[item]
        return X, t


# pytorch의 네트워크 클래스. nn.Module을 상속받고 모델 구조를 정의한 후 forward 메서드를 정의해준다.
class TwoLayerNet_pytorch(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.network1 = nn.Sequential(
        nn.Linear(self.input_size, self.hidden_size),
        nn.Sigmoid(),
        nn.Linear(self.hidden_size, self.output_size),
        nn.Sigmoid()
    )
  def forward(self, x):
    y = self.network1(x)
    return y

# hyperparameter 선언
epochs = 20000
learning_rate = 0.1

net = TwoLayerNet_pytorch(input_size=2, hidden_size=3, output_size=1) # 모델을 인스턴스로 만듦.
net.apply(weight_init) # 만들어진 모델 인스턴스에 weight 초기화 함수를 수행
optimizer = optim.SGD(net.parameters(), lr=learning_rate) # optimizer 선언.
dataloader = torch.utils.data.DataLoader(xor_dataloader(), batch_size=4) # dataloader 선언.

MSE = nn.MSELoss() # loss function 선언.

train_loss_list = [] # 결과 출력을 위한 코드
for epoch in range(epochs):
    for i, (X, t) in enumerate(dataloader,0):
        # 순전파
        Y = net(X)
        loss = MSE(Y, t)

        train_loss_list.append(loss) # 결과 출력을 위한 코드
         
        optimizer.zero_grad()  # optimizer의 gradient 텐서들을 초기화
        loss.backward() # backpropagation 수행. 각 weight에 대한 gradient 계산
        optimizer.step() # weight 갱신

# 학습종료 후 결과물 확인 (optional)
print("Input is")
print(X)
print("expected output is")
print(t)
print("actual output is ")
print(Y)
plt.plot(train_loss_list)